Preparing necessary imports

In [37]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from getpass import getpass
import omero.gateway as gw
import omero

from credentials import *

### Defining display functions and a tag_to_function mapping

In [38]:
def display_argoSIM_E(images):
    for image in images:
        print(f'Measurements for image id: {image.getId()}\n{image.getName()}')
        for ann in image.listAnnotations():
            if ann.OMERO_TYPE == omero.model.MapAnnotationI and \
                ann.getNs().startswith('metrics'):
                print('Key: value')
                for m in ann.getValue():
                    print(f'{m[0]}: {m[1]}')

tag_to_function = {'Argo-SIM_E_90deg': display_argoSIM_E,
                   'Argo-SIM_E_0deg': display_argoSIM_E}

### Open connection

In [39]:
conn = gw.BlitzGateway(username=USER,
                       passwd=PASSWORD,
                       host=HOST,
                       group=GROUP,
                       port=PORT)

# conn = gw.BlitzGateway(username=input('OMERO username:'), password=getpass('OMERO password:'),
#                        group=GROUP, host=OMERO_SERVER, port=PORT, secure=True)

conn.connect()
print(f'Connection successful: {conn.isConnected()}')

Connection successful: True


### Selecting the Microscope

In [40]:
microscopes = list(conn.getObjects('Project'))
micr_menu = widgets.Dropdown(options=[(m.getName(), i) for i, m in enumerate(microscopes)],
                     value=0,
                     description='Select a microscope:',
                     disabled=False,)
display(micr_menu)

Dropdown(description='Select a microscope:', options=(('microscope_2', 0), ('microscope_1', 1)), value=0)

### Selecting the Dataset

In [41]:
microscope = microscopes[micr_menu.value]

datasets = list(microscope.listChildren())
dataset_menu = widgets.Dropdown(options=[(d.getName(), i) for i, d in enumerate(datasets)],
                     value=0,
                     description='Select a dataset:',
                     disabled=False,)
display(dataset_menu)

Dropdown(description='Select a dataset:', options=(('2017-02-20_monthly', 0), ('2017-02-20_yearly', 1), ('2017…

### Select the type of analysis to display

In [44]:
dataset = datasets[dataset_menu.value]
images = list(dataset.listChildren())

# Get the relevant tags associated to the images in the dataset
relevant_tags = ['Argo-SIM_B', 'AArgo-SIM_E_0deg', 'Argo-SIM_E_90deg', 'PSF beads']
present_tags = []

for image in images:
    for ann in image.listAnnotations():
        if ann.OMERO_TYPE == omero.model.TagAnnotationI and \
                ann.getTextValue() in relevant_tags and \
                ann.getTextValue() not in present_tags:
            present_tags.append(ann.getTextValue())

tags_menu = widgets.Dropdown(options=[tag for tag in present_tags],
                             description='Select an analysis type:',
                             disabled=False,)
display(tags_menu)

Dropdown(description='Select an analysis type:', options=('Argo-SIM_E_90deg', 'Argo-SIM_B', 'PSF beads'), valu…

### Run visualization

In [45]:
images_to_display = []

for image in images:
    for ann in image.listAnnotations():
        if ann.OMERO_TYPE == omero.model.TagAnnotationI and \
                ann.getTextValue() == tags_menu.value:
            images_to_display.append(image)

tag_to_function[tags_menu.value](images_to_display)


Measurements for image id: 1
201702_RI510_Argolight-1-1_004_SIR_ALX_THR.ome.tif


TypeError: 'method' object is not iterable